In [31]:
# Imports
from pathlib import Path
Path.ls = lambda x: list(x.iterdir())
from pynwb import NWBFile
import tdt
import numpy as np

## Set Paths

In [ ]:
blockpath = '/Users/bendichter/Desktop/Chang/data/TDTBackup/EC125_B22';
imaging_path = '/Users/bendichter/Desktop/Chang/data/EC125/Imaging';
elecspath = fullfile(imaging_path, 'elecs', 'TDT_elecs_all.mat');
hilb_hg_path = '/Users/bendichter/Desktop/Chang/data/EC125/EC125_B22/HilbAA_70to150_8band';
 

In [3]:
blockpath = 'try2/B20_fixation/'
eyetrackerpath = 'try2/Eye_fv_subNS140_02.mat'
elecspath = '' # TODO: get elecs data structure


## Create NWB file

In [ ]:
[basepath, blockname] = fileparts(blockpath);

nwb_path = fullfile(basepath, [blockname '.nwb']);
%%
tdt = contrib.tdt.TDTbin2mat(blockpath);

%%
date = datevec([tdt.info.date tdt.info.utcStartTime]);

session_start_time = datetime(date,'Format','yyyy-MM-dd''T''HH:mm:SSZZ',...
    'TimeZone','local');

file = nwbfile('session_description', 'a test NWB File', ...
    'identifier', blockname, ...
    'session_start_time', session_start_time);

In [7]:
blockpath = Path(blockpath)
tdt_data = tdt.read_block(str("try2/B20_fixation"))

read from t=0s to t=472.71s


/home/dent/scratch/venv/lib/python3.7/site-packages/tdt/TDTbin2py.py:1684: Warning: Expecting SEV files for RSn1 but none were found, skipping...
  warnings.warn('Expecting SEV files for {0} but none were found, skipping...'.format(current_name), Warning)
/home/dent/scratch/venv/lib/python3.7/site-packages/tdt/TDTbin2py.py:1684: Warning: Expecting SEV files for RSn2 but none were found, skipping...
  warnings.warn('Expecting SEV files for {0} but none were found, skipping...'.format(current_name), Warning)


Using 24414.0625 Hz as SEV sampling rate for Wav5


In [12]:
nwb = NWBFile(session_description='test_reading_TDT_file',  # required
                  identifier='NS140_02',  # required
                  session_start_time=tdt_data.info.start_date)

## Create Electrode Table 

In [ ]:
%% Electrode Table

[file, ecog_channels] = elecs2ElectrodeTable(file, elecspath);

x = length(file.general_extracellular_ephys_electrodes.id.data);

ov = types.untyped.ObjectView('/general/extracellular_ephys/electrodes');

electrode_table_region = types.core.DynamicTableRegion('table', ov, ...
    'description', 'all electrodes',...
    'data', [1 x]');

In [13]:
# TODO figure out what the electrode table is and how to create it

## Read in ECOG

In [ ]:
%% ECoG

stream_names = fieldnames(tdt.streams);

ecog_stream_names = sort(stream_names(contains(stream_names, 'Wav')));

Data = [];
for i = 1:length(ecog_stream_names)
    stream = tdt.streams.(ecog_stream_names{i});
    Data = [Data, stream.data'];
end
Data = Data(:, ecog_channels);

In [40]:
# Here our data doesn't seem to correspond to the Chang Lab's data.
# Looks like they're appending their "Wav" files together, we don't have the correct dimensions
ecog_data = None
for k, v in tdt_data.streams.items():
    if "Wav5" in k:
        ecog_data = v.data.T


## Create electrical series

In [ ]:
es = types.core.ElectricalSeries('starting_time', stream.startTime,...
    'starting_time_rate', stream.fs,...
    'data',Data',...
    'electrodes', electrode_table_region,...
    'data_unit', 'V');

file.acquisition.set('ECoG', es);


ECoG_soft_link = types.untyped.SoftLink('/aquisition/ECoG');

In [48]:
from pynwb.ecephys import ElectricalSeries
if ecog_data is not None:
    
    es = ElectricalSeries(name="WAV files",
                     data = ecog_data,
                     electrodes = None # TODO: figure out electrodes, does not run
                    )
    NWBFile.add_acquisition(es)

TypeError: ElectricalSeries.__init__: None is not allowed for 'electrodes' (expected 'DynamicTableRegion', not None)

## Create Eyetracking

In [ ]:
%% ANIN

stream = tdt.streams.ANIN;

labels = {'microphone', 'speaker1', 'speaker2', 'anin4'};

for i = 1:length(labels)
    ts = types.core.TimeSeries('starting_time',stream.startTime,...
        'starting_time_rate',stream.fs,...
        'data',stream.data(i,:)',...
        'data_unit','V?');
    file.acquisition.set(labels{i}, ts);
end

In [ ]:
%% Cortical Surface
% generateExtension('/Users/bendichter/dev/nwbext_ecog/nwbext_ecog/ecog.namespace.yaml');

mesh_dir = fullfile(imaging_path, 'Meshes');
mesh_file_list = dir([mesh_dir '/*_pial.mat']);
mesh_file_list = {mesh_file_list.name};

cortical_surfaces = types.ecog.CorticalSurfaces;

for i = 1:length(mesh_file_list)
    mesh_file = mesh_file_list{i};
    surf_load = load(fullfile(mesh_dir, mesh_file));
    if isfield(surf_load,'mesh')
        faces = surf_load.mesh.tri - 1; % faces stored as 0-indexed in nwbext_ecog
        vertices = surf_load.mesh.vert;
    elseif isfield(surf_load,'cortex')
        faces = surf_load.cortex.tri - 1; % faces stored as 0-indexed in nwbext_ecog
        vertices = surf_load.cortex.vert;
    else
        keyboard
    end
    surf = types.ecog.Surface('faces', faces', 'vertices', vertices');
    surf_name = mesh_file(find(mesh_file == '_', 1)+1 : end-9);
    cortical_surfaces.surface.set(surf_name, surf);
end

file.acquisition.set('CorticalSurfaces', cortical_surfaces);